<a href="https://colab.research.google.com/github/paulcodrea/reddit_humor/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
# import os
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# from time import time
# from keras.models import Sequential, load_model
# from keras.layers.core import Dense, Dropout 
# from keras.layers import LSTM
# from time import time
# from keras.callbacks import EarlyStopping
# from sklearn.preprocessing import MinMaxScaler
# from pathlib import Path

# import nltk
# import regex as re
# from collections import defaultdict

from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
# from nltk.stem.snowball import EnglishStemmer

In [24]:
# We might have to change the following

config = {
    "learning_rate": 0.001,
    "epochs": 40, 
    "batch_size": 4,
    "train_p": 0.55,
    "val_p": 0.05,
    "LSTM_layer": [50, 100],
    "Dropout_layer": [0.15, 0.2],
    "activation": 'tanh',
    "timesteps": 1,
}

In [61]:
class LSTM_model:
    def __init__(self, path):
        self.path = path # Path to the dataset
        self.data = pd.DataFrame() # Dataframe to store the dataset

        self.context_window = 3 # Context window size
        self.w2v_feature_vector = []

    def read_dataset(self):
        """
        Reads the dataset from the given path.
        """
        ret = pd.read_csv(self.path)
        ret.drop(columns=['Unnamed: 0'], inplace=True)
        
        return ret

    def preprocess_text(self):
        """
        Preprocesses the text data.
        """
        self.data['tokens'] = self.data['joke'].apply(word_tokenize) # tokenize the text but keep the punctuation


    def construct_word2vec(self):
        """
        Constructs the word2vec model. (Feature vector)
        """
        self.w2v_feature_vector = []
        context_words = [] # Construct window list for word2vec
        
        for line in self.data['tokens']:
            for index, word in enumerate(line):
                if self.context_window > 0:
                    left = index - self.context_window//2
                    right = index + self.context_window//2 + 1
                else:
                    left = index - self.context_window//2
                    right = index + self.context_window//2
                context_words.append([line[i] for i in range(left, right) if i >= 0 and i < len(line)])
 

        # Create a word2vec model
        # context_words = [['a', 'b'], ['a', 'b', 'c'], ['b', 'c', 'd'], ['c', 'd', 'e'], ['d', 'e']] -> list of lists of words and window size is 5
        # vector_size = 50 -> dimension of the feature vector (pairs)
        # min_count = 4 -> minimum number of occurrences of a word in the corpus
        # workers = 4 -> number of threads to use
        # window = 5 -> window size
        model = Word2Vec(context_words, window=self.context_window, workers=4)

        for line in self.data['tokens']:
            for index, word in enumerate(line):
                if word in model.wv.key_to_index:
                    self.w2v_feature_vector.append(model.wv.get_vector(word))
                else:
                    self.w2v_feature_vector.append(np.zeros(10))


In [62]:
# SETTINGS for local machine - change this for Goolg Colab
path = "dataset/final_jokes(1283).csv" #"/content/drive/MyDrive/NLU_Humor-detection/final_jokes(1283).csv"

joke_model = LSTM_model(path)
joke_model.data = joke_model.read_dataset()
joke_model.preprocess_text()
joke_model.construct_word2vec()

print(joke_model.data['tokens'][0])
print(joke_model.w2v_feature_vector[0])

['My', '5', 'year', 'old', 'just', 'ran', 'out', 'of', 'her', 'room', 'to', 'tell', 'me', 'this', 'joke', 'she', 'just', 'thought', 'up', ':', 'what', 'did', 'the', 'cow', 'say', 'after', 'he', 'was', 'fed', '?', 'Moooooooooore', '!', 'I', "'ve", 'never', 'been', 'this', 'proud', 'of', 'anything', 'in', 'my', 'life', '.']
[-0.03717059  0.18439339  0.06661823  0.01248952  0.06323018 -0.47885564
  0.08743261  0.58742625 -0.28815296 -0.27545375  0.04169046 -0.462599
  0.03613111  0.2198521   0.04350519 -0.17073017  0.15139964 -0.04017985
 -0.00729127 -0.6064894   0.22195981  0.07907391  0.21248852 -0.07872106
 -0.16553278  0.08017829 -0.12476471 -0.05605224 -0.24086802  0.10278188
  0.30119467  0.00233689  0.10685571 -0.33005694 -0.17532691  0.34755224
  0.23784187 -0.23665161 -0.1492309  -0.47725475  0.06649645 -0.38529077
 -0.16209906 -0.0437514   0.2908809   0.03840032 -0.14959303  0.01834923
  0.11771975  0.02263468  0.09958069 -0.26267114  0.00667423  0.01212476
 -0.08166009  0.14622